In [1]:
import os
execfile(os.path.join(os.environ["SPARK_HOME"], 'python/pyspark/shell.py'))

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.1.1
      /_/

Using Python version 2.7.12 (default, Nov 19 2016 06:48:10)
SparkSession available as 'spark'.


In [2]:
from pyspark.sql import SparkSession
sparkSession = SparkSession.builder.enableHiveSupport().master("local [2]").getOrCreate()

In [3]:
data = sparkSession.read.parquet("/data/sample264")
meta = sparkSession.read.parquet("/data/meta")

In [4]:
import pyspark.sql.functions as f
from pyspark.sql import Window

In [62]:
window1 = Window.partitionBy("userId").orderBy("timestamp").rangeBetween(-420,420)

In [63]:
similar = data.withColumn("similars", f.collect_list("trackId").over(window1)) \
              .withColumn("track2", f.explode("similars")) \
              .filter('trackId != track2') \
              .select(f.col('trackId').alias('track1'), 'track2') \
              .select((f.least(f.col('track1'),f.col('track2'))).alias('track1'),
                      (f.greatest(f.col('track1'),f.col('track2'))).alias('track2')) \
              .groupBy('track1', 'track2').count().cache()

In [66]:
window2 = Window.partitionBy("track1").orderBy(f.col("count").desc())
window2_a = Window.partitionBy("track1")

In [67]:
normalized = similar.withColumn("row_number", f.row_number().over(window2)) \
                    .filter("row_number<=40") \
                    .withColumn("sum_count", f.sum(f.col("count")).over(window2_a)) \
                    .withColumn("norm_count", f.col("count")/f.col("sum_count")*0.5) \
                    .select("track1", "track2", "norm_count")

In [68]:
result = normalized.orderBy(f.col("norm_count").desc(), "track1", "track2").take(40)

In [69]:
for row in result:
    print '{0} {1}'.format(row[0], row[1])

798256 923706
798319 837992
798322 876562
798331 827364
798335 840741
798374 816874
798375 810685
798379 812055
798380 840113
798396 817687
798398 926302
798405 867217
798443 905923
798457 918918
798460 891840
798461 940379
798470 840814
798474 963162
798477 883244
798485 955521
798505 905671
798545 949238
798550 936295
798626 845438
798691 818279
798692 898823
798702 811440
798704 937570
798725 933147
798738 894170
798745 799665
798782 956938
798801 950802
798820 890393
798833 916319
798865 962662
798931 893574
798946 946408
799012 809997
799024 935246
